MVCC (Multi-Version Concurrency Control) — это механизм управления параллельным доступом к данным в базе данных,  
который широко используется в PostgreSQL и других СУБД для поддержки одновременных транзакций.
Concurrency - параллелизм, параллельность

Он позволяет разным транзакциям видеть базу данных в разных «версиях» (или состояниях), что обеспечивает высокую степень изоляции и консистентности данных.

Механизм работает следующим образом:

Создание версий данных: когда транзакция вносит изменения в базу данных, PostgreSQL не изменяет существующие строки, а создаёт новые версии строк с обновленными данными.

Версионирование по времени: каждая версия строки имеет информацию о времени начала и окончания её действия.

Уровни изоляции: MVCC позволяет разным транзакциям работать на разных уровнях изоляции.

Удаление устаревших данных: PostgreSQL автоматически удаляет устаревшие версии данных, когда транзакция, которая их создала, успешно завершается.

ACID - atomicity, consistency, isolation, durability

CAP - Consistency, Availability, Partition tolerance  
Теорема CAP (также известная как теорема Брюера) — эвристическое утверждение о том,  
что в любой реализации распределённых вычислений возможно обеспечить не более двух из трёх следующих свойств:  
1. Согласованность данных: во всех вычислительных узлах в один момент времени данные не противоречат друг другу.  
2. Доступность: любой запрос к распределённой системе завершается откликом, однако без гарантии, что ответы всех узлов системы совпадают.  
3. Устойчивость к фрагментации: расщепление распределённой системы на несколько изолированных секций не приводит к некорректности отклика от каждой из секций.  

# Оптимизация запросов

Использование предложения IN иногда может привести к неоптимальной производительности, особенно при работе с большими наборами данных.  
Замена предложения IN подзапросом EXISTS или операцией JOIN может помочь оптимизировать запрос SQL,  
позволяя ядру базы данных лучше использовать индексы и другие методы оптимизации.

Оператор EXISTS в SQL позволяет проверить, возвращает ли подзапрос какое-либо значение.

Формальный синтаксис оператора: WHERE [NOT] EXISTS (подзапрос).

Пример:

найдем всех покупателей из таблицы Customer, которые делали заказы


In [ ]:
SELECT
    * 
FROM
    Customers
WHERE
    EXISTS (SELECT
                * 
            FROM 
                Orders 
            WHERE 
                Orders.CustomerId = Customers.Id);

найдем все товары из таблицы Products, на которые не было заказов в таблице Orders

In [ ]:

SELECT
* 
FROM
    Products 
WHERE 
    NOT EXISTS (SELECT
                    * 
                FROM
                    Orders 
                WHERE Products.Id = Orders.ProductId).

Материализованные представления хранят результат запроса и могут использоваться для кэширования результатов сложных вычислений или агрегатов, 
к которым часто обращаются, но редко обновляют. Использование материализованных представлений может привести
к значительному повышению производительности при рабочих нагрузках с большим объемом чтения

Что такое план и зачем он нужен
То, что база нам готова рассказать - это план выполнения запроса, который представляет из себя некоторую древовидную структуру в виде либо текста,  
либо (в зависимости от выбранного формата) JSON, XML или YAML.

План выполнения показывает, как будут сканироваться таблицы, затрагиваемые оператором — просто последовательно, по индексу и т. д.  
— а если запрос связывает несколько таблиц, какой алгоритм соединения будет выбран для объединения считанных из них строк.

Чтобы получить от базы план запроса, как и любую другую вещь в PostgreSQL, мы должны выполнить определенную команду - EXPLAIN:

In [ ]:
EXPLAIN [ ( параметр [, ...] ) ] оператор
EXPLAIN [ ANALYZE ] [ VERBOSE ] оператор

Здесь допускается параметр:

    ANALYZE [ boolean ]
    VERBOSE [ boolean ]
    COSTS [ boolean ]
    SETTINGS [ boolean ]
    GENERIC_PLAN [ boolean ]
    BUFFERS [ boolean ]
    WAL [ boolean ]
    TIMING [ boolean ]
    SUMMARY [ boolean ]
    FORMAT { TEXT | XML | JSON | YAML }

Фактически выводятся два числа: стоимость запуска до выдачи первой строки и общая стоимость выдачи всех строк.

С параметром ANALYZE оператор будет выполнен на самом деле, а не только запланирован.   
При этом в вывод добавляются фактические сведения о времени выполнения, включая общее время, затраченное на каждый узел плана (в миллисекундах)     
и общее число строк, выданных в результате.     
Это помогает понять, насколько близки к реальности предварительные оценки планировщика.  

VERBOSE  
Вывести дополнительную информацию о плане запроса.    
возвращаемые узлом столбцы, то есть будет строчка output и названия столбцов, которые вернет узел

COSTS  
Вывести рассчитанную стоимость запуска и общую стоимость каждого узла плана, а также рассчитанное число строк и ширину каждой строки.   
Этот параметр по умолчанию равен TRUE.

SUMMARY  
Включить сводку (например, суммарное время) после плана запроса.   
Сводка включается по умолчанию, когда используется ANALYZE, но этот параметр позволяет получить её и с другими вариантами команды.

FORMAT  
Установить один из следующих форматов вывода: TEXT, XML, JSON или YAML.   
Последние три формата содержат ту же информацию, что и текстовый, но больше подходят для программного разбора. По умолчанию выбирается формат TEXT.

In [ ]:
EXPLAIN (FORMAT JSON) SELECT * FROM foo;
           QUERY PLAN
--------------------------------
 [                             +
   {                           +
     "Plan": {                 +
       "Node Type": "Seq Scan",+
       "Relation Name": "foo", +
       "Alias": "foo",         +
       "Startup Cost": 0.00,   +
       "Total Cost": 155.00,   +
       "Plan Rows": 10000,     +
       "Plan Width": 4         +
     }                         +
   }                           +
 ]